In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import bibtexparser
import requests
from urllib.parse import quote

In [2]:
with open('seedPapers.bib') as bibtex_file:
  bib_database = bibtexparser.load(bibtex_file)
papers = bib_database.entries

In [16]:
desired_capabilities = DesiredCapabilities.FIREFOX
desired_capabilities["pageLoadStrategy"] = "none"

In [3]:
firefox_options = Options()
firefox_options.add_argument("--headless")
firefox_options.set_preference('permissions.default.stylesheet', 2)
firefox_options.set_preference('permissions.default.image', 2)
firefox_options.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')

driver = webdriver.Firefox(options=firefox_options, service=FirefoxService(GeckoDriverManager().install()))
  

In [42]:
# for count, paper in enumerate(papers):
#   if count > 1:
#     break
#   doi = paper['doi']
#   driver.get("https://www.webofscience.com/wos/woscc/basic-search")
#   # /html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-search-home/div[2]/div/app-input-route/app-search-basic/app-search-form/form/div[1]/app-search-row/div/div[1]/app-select-search-field/wos-select
#   # Select "DOI" search option
#   dropdown = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.XPATH, '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-search-home/div[2]/div/app-input-route/app-search-basic/app-search-form/form/div[1]/app-search-row/div/div[1]/app-select-search-field/wos-select'))
#   )
#   dropdown.click()
  
#   # Wait for the "DOI" option to be present and click it
#   doi_option = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.XPATH, '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-search-home/div[2]/div/app-input-route/app-search-basic/app-search-form/form/div[1]/app-search-row/div/div[1]/app-select-search-field/wos-select/div/div[1]/div/div[19]'))
#   )
#   doi_option.click()

#   # Enter the DOI in the search box
#   WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, search_path))).send_keys(doi)
  
#   # Click the search button
#   WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, search_botton))).click()
  
#   # Click the DOI button
#   time.sleep(3)
  
#   # Click on the first search result
#   result = driver.find_element(By.CSS_SELECTOR, 'div.search-results-item')
#   result.click()
  
#   time.sleep(5)  # Wait for the paper details to load

#   # Click on "Full Text at Publisher" link
#   full_text_link = driver.find_element(By.LINK_TEXT, 'Full Text at Publisher')
#   full_text_link.click()
  
#   time.sleep(5) # Wait for the full text to load
  
#   body_text = driver.find_element(By.TAG_NAME, 'body').text
#   print(body_text)

In [19]:

for paper in papers:
    doi = paper['doi']
    
    # Initialize the WebDriver for Firefox
    driver = webdriver.Firefox()

    # Open the website
    driver.get('https://www.webofscience.com/wos/woscc/basic-search')

    # Define the XPaths

    cookie_preferences_xpath = '//*[@id="onetrust-pc-btn-handler"]'

    confirm_choices_xpath = "//*[@class='save-preference-btn-handler onetrust-close-btn-handler']"

    # Confirm cookies option
    try:
        # Wait for the "Manage cookie preferences" button to be clickable and click it
        cookie_preferences_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, cookie_preferences_xpath))
        )
        cookie_preferences_button.click()

        # Wait for the "Confirm my choices" button to be clickable and click it
        confirm_choices_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, confirm_choices_xpath))
        )
        confirm_choices_button.click()
        
    except TimeoutException:
        print("Timeout while waiting for elements to be clickable.")
        
    except ElementClickInterceptedException:
        print("ElementClickInterceptedException occurred while clicking elements.")
        

    # exit the popup
    popup_x = "//*[@id='pendo-close-guide-e0e49982']"
    popup = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, popup_x))
    )
    popup.click()

    # Select DOI search option
    dropdown_xpath = "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-search-home/div[2]/div/app-input-route/app-search-basic/app-search-form/form/div[1]/app-search-row/div/div[1]/app-select-search-field/wos-select/button"

    # Wait for the search option box to be clickable and click it
    dropdown = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, dropdown_xpath))
    )

    dropdown.click()

    doi_option = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-search-home/div[2]/div/app-input-route/app-search-basic/app-search-form/form/div[1]/app-search-row/div/div[1]/app-select-search-field/wos-select/div/div[1]/div/div[19]"))
    )
    doi_option.click()

    # Enter the DOI in the search box
    search_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "input[data-ta='search-criteria-input']"))
)
    search_input.send_keys(doi)
    
    # click the search button
    search_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-ta='run-search']"))
)

    # Click the Search button
    search_button.click()
    
    # Click full text at publisher
    full_text = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.LINK_TEXT, "Full Text at Publisher"))
    ).click()
    
    # copy and paste the main content of the paper
    


NameError: name 'TimeoutException' is not defined

In [52]:

for paper in papers:
    doi = paper['doi']
    
    # Initialize the WebDriver for Firefox
    driver = webdriver.Firefox()

    # Open openAlex website
    driver.get('https://openalex.org/')

    



KeyboardInterrupt: 

In [59]:
# for paper in papers:
#   openalex_base = "https://api.openalex.org/works/"
#   doi = "https://doi.org/" + quote(paper['doi'])
  
#   openalex = requests.get(openalex_base + doi)
#   best_oa_location = openalex.json()
  
#   print(best_oa_location["best_oa_location"])

In [58]:
# for paper in papers:
#   # openalex_base = "https://api.openalex.org/works/"
#   doi = "https://doi.org/" + quote(paper['doi'])
  
#   openalex = requests.get(doi)
#   best_oa_location = openalex.json()
  
#   print(best_oa_location.keys())

In [4]:
# from selenium.webdriver.firefox.firefox_profile import FirefoxProfile

# Load the default Firefox profile
profile = webdriver.FirefoxProfile()

# Set the preference for cookie behavior
# profile.set_preference("network.cookie.cookieBehavior", 2)


options = Options()
options.add_argument("--disable-notifications")
options.add_argument("--headless")


for count, paper in enumerate(papers):
  doi = "https://doi.org/" + quote(paper['doi'])
  # doi = "https://link.springer.com/10.1007/s40572-020-00291-4"
  driver = webdriver.Firefox(
    options=options,
    # firefox_profile=profile,
    executable_path=GeckoDriverManager().install()
  )
  
  driver.get(doi)
  time.sleep(5)
  
  
  # try: 
  #   main_content = driver.find_element(By.ID, 'main-content')
  
  
  # driver.add_cookie({"name": "cookie_consent", "value": "accepted"})
  # driver.refresh()
  # time.sleep(5)
  # alert = driver.switch_to.alert
  
  # # Accept the alert
  # alert.accept()

/var/folders/2z/mw_brprd0jbb6fw34zr7m2bw0000gn/T/ipykernel_1222/1268132420.py:4: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  profile = webdriver.FirefoxProfile()
/var/folders/2z/mw_brprd0jbb6fw34zr7m2bw0000gn/T/ipykernel_1222/1268132420.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(


KeyboardInterrupt: 

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options

def extract_full_text(url):
    # Initialize Selenium WebDriver for Firefox
    options = Options()
    options.add_argument('--headless')  # Run in headless mode for performance
    driver = webdriver.Firefox(options=options, service=FirefoxService(GeckoDriverManager().install()))


    # Open the webpage
    driver.get(url)

    try:
        # Locate the "Abstract" section by finding an element with text "Abstract"
        abstract_element = driver.find_element(By.XPATH, "//h2[contains(text(), 'Abstract') or contains(text(), 'ABSTRACT') or contains(text(), 'abstract')]")

        # Get the parent container of the Abstract section
        parent_container = abstract_element.find_element(By.XPATH, "..")

        # Traverse up the tree to find the main container
        while parent_container:
            if parent_container.get_attribute('class'):
                # Check if the class name suggests it contains the full text (e.g., "full text", "full content")
                class_name = parent_container.get_attribute('class').lower()
                if 'full' in class_name and ('text' in class_name or 'content' in class_name):
                    break
            parent_container = parent_container.find_element(By.XPATH, "..")

        # Expand all elements within the main container if needed
        # Use a simpler JavaScript execution to avoid cyclic references
        driver.execute_script("arguments[0].style.display = 'block';", parent_container)

        # Extract the text from the main container
        full_text = parent_container.text
        return full_text

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

    finally:
        # Close the WebDriver
        driver.quit()

# Example usage
url = 'https://journals.sagepub.com/doi/full/10.1177/0022343311431288'  # Replace with your URL
full_text = extract_full_text(url)
print(full_text)


WebDriverException: Message: Service /Users/bowenyi/.wdm/drivers/geckodriver/mac64/v0.35.0/geckodriver unexpectedly exited. Status code was: -9


In [3]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.common.by import By
from webdriver_manager.firefox import GeckoDriverManager
import time

def extract_full_text(url):
    # Setup Firefox options
    options = Options()
    options.add_argument('--headless')  # Comment this line to run in non-headless mode
    options.set_preference("devtools.console.stdout.content", True)
    options.set_preference("dom.max_script_run_time", 60)

    # Initialize the WebDriver
    driver = webdriver.Firefox(options=options, service=FirefoxService(GeckoDriverManager().install()))

    # Set page load timeout
    driver.set_page_load_timeout(30)

    try:
        # Open the webpage
        driver.get(url)
        time.sleep(5)  # Allow time for the page to load

        try:
            # Locate the "Abstract" section by finding an element with text "Abstract"
            abstract_element = driver.find_element(By.XPATH, "//h2[contains(text(), 'Abstract') or contains(text(), 'ABSTRACT') or contains(text(), 'abstract')]")

            # Get the parent container of the Abstract section
            parent_container = abstract_element.find_element(By.XPATH, "..")

            # Traverse up the tree to find the main container
            while parent_container:
                if parent_container.get_attribute('class'):
                    # Check if the class name suggests it contains the full text (e.g., "full text", "full content")
                    class_name = parent_container.get_attribute('class').lower()
                    if 'full' in class_name and ('text' in class_name or 'content' in class_name):
                        break
                parent_container = parent_container.find_element(By.XPATH, "..")

            # Expand all elements within the main container if needed
            # Use a simpler JavaScript execution to avoid cyclic references
            driver.execute_script("arguments[0].style.display = 'block';", parent_container)

            # Extract the text from the main container
            full_text = parent_container.text
            return full_text

        except Exception as e:
            print(f"Error while extracting full text: {e}")
            return ""

    finally:
        # Quit the driver
        driver.quit()

# Example usage
url = 'https://journals.sagepub.com/doi/full/10.1177/0022343311431288'  # Replace with your URL
full_text = extract_full_text(url)
print(full_text)


Error while extracting full text: Message: Cyclic object value
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:187:5
JavaScriptError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:457:5
cloneObject@chrome://remote/content/marionette/json.sys.mjs:43:11
cloneJSON@chrome://remote/content/marionette/json.sys.mjs:177:12
cloneObject@chrome://remote/content/marionette/json.sys.mjs:56:24
cloneJSON@chrome://remote/content/marionette/json.sys.mjs:177:12
cloneObject@chrome://remote/content/marionette/json.sys.mjs:56:24
cloneJSON@chrome://remote/content/marionette/json.sys.mjs:177:12
json.clone@chrome://remote/content/marionette/json.sys.mjs:180:10
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:187:25




In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def extract_full_text(url):
    # Initialize Selenium WebDriver (using Chrome in this example)
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run in headless mode for performance
    driver = webdriver.Chrome(options=options)

    # Open the webpage
    driver.get(url)
    time.sleep(5)  # Allow time for the page to load

    
    try:
        # Locate the "Abstract" section by finding an element with text "Abstract"
        abstract_element = driver.find_element(By.XPATH, "//h2[contains(text(), 'Abstract') or contains(text(), 'ABSTRACT') or contains(text(), 'abstract')]")

        # Get the parent container of the Abstract section
        parent_container = abstract_element.find_element(By.XPATH, "ancestor-or-self::div[1]")

        # Traverse up the tree to find the main container
        while parent_container:
            if parent_container.get_attribute('class'):
                # Check if the class name suggests it contains the full text (e.g., "full text", "full content")
                class_name = parent_container.get_attribute('class').lower()
                print(f"Checking class: {class_name}")  # Debugging output
                if 'full' in class_name and ('text' in class_name or 'content' in class_name):
                    break
            parent_container = parent_container.find_element(By.XPATH, "ancestor::div[1]")
        
        # Check if parent_container was found
        if parent_container is None:
            raise Exception("Could not find the main content container.")

        # Expand all elements within the main container if needed
        driver.execute_script("arguments[0].style.display = 'block';", parent_container)

        # Extract the text from the main container
        full_text = parent_container.text
        return full_text

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

    finally:
        # Close the WebDriver
        driver.quit()

# Example usage
url = 'https://journals.sagepub.com/doi/full/10.1177/0022343311431288'
full_text = extract_full_text(url)
print(full_text)


Checking class: core-container
An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"ancestor::div[1]"}
  (Session info: chrome-headless-shell=127.0.6533.99); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5561c28106aa <unknown>
#1 0x5561c24e1600 <unknown>
#2 0x5561c2530bcb <unknown>
#3 0x5561c2530eb1 <unknown>
#4 0x5561c25256a6 <unknown>
#5 0x5561c25538cd <unknown>
#6 0x5561c2525599 <unknown>
#7 0x5561c2553a6e <unknown>
#8 0x5561c257204a <unknown>
#9 0x5561c2553643 <unknown>
#10 0x5561c2523d31 <unknown>
#11 0x5561c252479e <unknown>
#12 0x5561c27d823b <unknown>
#13 0x5561c27dc1d2 <unknown>
#14 0x5561c27c55f5 <unknown>
#15 0x5561c27dcd62 <unknown>
#16 0x5561c27aa23f <unknown>
#17 0x5561c27ffe48 <unknown>
#18 0x5561c2800020 <unknown>
#19 0x5561c280f47c <unknown>
#20 0x152ebda861ca start_thread

None


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def extract_full_text(url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(5)

    try:
        # Locate the "Abstract" section
        abstract_element = driver.find_element(By.XPATH, "//h2[contains(text(), 'Abstract') or contains(text(), 'ABSTRACT') or contains(text(), 'abstract')]")
        if abstract_element:
            # Traverse the DOM tree upwards from the Abstract section
            parent_container = abstract_element.find_element(By.XPATH, "ancestor-or-self::div[1]")
            while parent_container:
                class_name = parent_container.get_attribute('class').lower()
                if 'content' in class_name or 'paper' in class_name or 'article' in class_name:
                    break
                parent_container = parent_container.find_element(By.XPATH, "ancestor::div[1]")

            # Check for common content containers
            if parent_container is None:
                parent_container = driver.find_element(By.XPATH, "//div[contains(@class, 'main-content') or contains(@class, 'article-body') or contains(@class, 'document-body')]")

            # Expand all elements within the main container if needed
            driver.execute_script("arguments[0].style.display = 'block';", parent_container)
            full_text = parent_container.text
            return full_text
        else:
            raise Exception("Could not find the Abstract section.")
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    finally:
        driver.quit()

# Example usage
url = 'https://journals.sagepub.com/doi/full/10.1177/0022343311431288'
full_text = extract_full_text(url)
print(full_text)

An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"ancestor::div[1]"}
  (Session info: chrome-headless-shell=127.0.6533.99); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x559ae08e26aa <unknown>
#1 0x559ae05b3600 <unknown>
#2 0x559ae0602bcb <unknown>
#3 0x559ae0602eb1 <unknown>
#4 0x559ae05f76a6 <unknown>
#5 0x559ae06258cd <unknown>
#6 0x559ae05f7599 <unknown>
#7 0x559ae0625a6e <unknown>
#8 0x559ae064404a <unknown>
#9 0x559ae0625643 <unknown>
#10 0x559ae05f5d31 <unknown>
#11 0x559ae05f679e <unknown>
#12 0x559ae08aa23b <unknown>
#13 0x559ae08ae1d2 <unknown>
#14 0x559ae08975f5 <unknown>
#15 0x559ae08aed62 <unknown>
#16 0x559ae087c23f <unknown>
#17 0x559ae08d1e48 <unknown>
#18 0x559ae08d2020 <unknown>
#19 0x559ae08e147c <unknown>
#20 0x146bf58241ca start_thread

None


In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def extract_full_text(url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(5)

    try:
        # Locate the "Abstract" section
        abstract_element = driver.find_element(By.XPATH, "//h2[contains(text(), 'Abstract') or contains(text(), 'ABSTRACT') or contains(text(), 'abstract')]")
        if abstract_element:
            # Find the main content container
            # print("yes1")
            main_container = None
            for element in driver.find_elements(By.XPATH, "//*[contains(@class, 'content') or contains(@class, 'article') or contains(@class, 'paper') or contains(@class, 'full') or contains(@class, 'main-content') or contains(@class, 'article-body') or contains(@class, 'document-body')]"):
                if element.text and 'Abstract' in element.text:
                    main_container = element
                    # print("yes2")
                    break

            if main_container:
                # Expand all elements within the main container if needed
                driver.execute_script("arguments[0].style.display = 'block';", main_container)
                full_text = main_container.text
                return full_text
            else:
                raise Exception("Could not find the main content container.")
        else:
            raise Exception("Could not find the Abstract section.")
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    finally:
        driver.quit()

# Example usage
url = 'https://journals.sagepub.com/doi/full/10.1177/0022343311431288'
full_text = extract_full_text(url)
print(full_text)

yes1
yes2
Journal of Peace Research
Impact Factor: 3.4
5-Year Impact Factor: 4.5
Available access
Introduction
First published online January 31, 2012
Whither the weather? Climate change and conflict
Nils Petter GleditschView all authors and affiliations
Volume 49, Issue 1
https://doi.org/10.1177/0022343311431288
Contents
PDF / ePub
More
Abstract
Until recently, most writings on the relationship between climate change and security were highly speculative. The IPCC assessment reports to date offer little if any guidance on this issue and occasionally pay excessive attention to questionable sources. The articles published in this special issue form the largest collection of peer-reviewed writings on the topic to date. The number of such studies remains small compared to those that make up the natural science base of the climate issue, and there is some confusion whether it is the effect of ‘climate’ or ‘weather’ that is being tested. The results of the studies vary, and firm conclusions 